## Obtain Data

In this notebook we perform the following steps:
* Establish the first hour of the dataset
* For the first month,
  * Obtain a list of available stations by state
  * Obtain temperature observations from weather stations in the MISO footprint
    * Stations are organized into MISO regions by state boundaries
    * Stations are predominantly clustered in population centers, making many observation redundant
    * There are many lacunae in some stations
  * Obtain the actual hourly MISO Load data and historical Medium-Term Load Forecasts (MTLF)
  * Join Load and MTLF data with weather observations to complete the raw data
  * Persist the data and demonstrate use of dataset wrapper class
* Update the dataset to the current day
* Identify and mitigate lacunae
* Publish the data

### Define Date Ranges

In [2]:
%load_ext autoreload
%autoreload 2

from datetime import datetime, timezone, timedelta
from weather_data import prevailing_time as est

# beginning of MISO's historical records that include the southern region (zones 8-10)
# five hours weather data was lost for 2015-06-17
first_hour = est(2015, 6, 18, 4)

# latest date with actual load data available is
# l = date.today() - timedelta(days=2)
# last_hour = est(l.year, l.month, l.day, 23)
# instead, fix the date for repeatbility
last_hour = est(2022, 4, 22, 23)

test_split = last_hour - timedelta(days=364, hours=23)
validation_split = test_split - timedelta(days=365)


### Obtain Weather Data for Zones

[Iowa State ASOS Network Downloads](https://mesonet.agron.iastate.edu/request/download.phtml)

In [3]:
from weather_data import get_station_df
from pathlib import Path
from os.path import isfile
import pandas as pd

zones = { 1 : {'MN': ['MSP',  # Minneapolis / St. Paul (STP)
                      'RST',  # Rochester
                      'DYT'], # Duluth
               'ND': ['FAR',  # Fargo
                      'BIS',  # Bismarck
                      'GFK'], # Grand Forks
               'SD': ['ABR'], # Aberdeen
               'WI': ['LSE'], # La Crosse
               'IL': ['SFY']  # Savanna 
              },
          2 : {'WI': ['MSN', 'MKE', 'EAU', 'GRB'],
               'MI': ['ANJ', 'SAW', 'IWD']},
          3 : {'IA': ['DSM', 'CID', 'DVN', 'SUX', 'ALO', 'MCW']}
        }

def download_file_path(zone, state, station):
    zone_data = f"./data/zone_{zone}"
    Path(zone_data).mkdir(exist_ok=True)
    return f'{zone_data}/{state}_{station}.parquet'

def download_station(zone, state, station):
    path = download_file_path(zone, state, station) 
    if isfile(path):
        return pd.read_parquet(path)

    station = get_station_df(station, first_hour, last_hour)
    if station is None:
        print(f'Retrieve {station} failed')
        return None
    return station.to_parquet(path)

from multiprocessing import cpu_count
from joblib import Parallel, delayed
def do_parallel(func):
    parallel = Parallel(n_jobs=cpu_count())
    result = {}
    for zone in zones:
        stations = [(state, station) for state in zones[zone].keys() for station in zones[zone][state]]
        result[zone] = parallel(delayed(func)(zone, state, station) for (state, station) in stations)
    return result

In [4]:
_ = do_parallel(download_station)

We have obtained raw weather observations at various intervals, usually 15 minutes, but there is a lot of missing data.

In [5]:
raw_df = pd.DataFrame()
for zone in zones:
  for state in zones[zone]:
      for station in zones[zone][state]:
        raw_df = pd.concat([raw_df, pd.read_parquet(download_file_path(zone, state, station))])
raw_df[raw_df['tmpf'] == 'M'].head()

,station,valid,tmpf,lat,lon,feel
9559,MSP,2016-04-30 17:00,M,44.8854,-93.2313,M
9560,MSP,2016-04-30 17:15,M,44.8854,-93.2313,M
9561,MSP,2016-04-30 17:20,M,44.8854,-93.2313,M
9562,MSP,2016-04-30 17:35,M,44.8854,-93.2313,M
9564,MSP,2016-04-30 18:00,M,44.8854,-93.2313,M


In [6]:
raw_df[raw_df['tmpf'] == 'M'].shape

(10880084, 6)

In [9]:
raw_df['temp'] = pd.to_numeric(raw_df['tmpf'], errors='coerce')
raw_df.head()

,station,valid,tmpf,lat,lon,feel,temp
0,MSP,2015-06-18 00:53,68.00,44.8854,-93.2313,68.00,68.00
1,MSP,2015-06-18 01:53,66.92,44.8854,-93.2313,66.92,66.92
2,MSP,2015-06-18 02:27,66.92,44.8854,-93.2313,66.92,66.92
3,MSP,2015-06-18 02:48,66.20,44.8854,-93.2313,66.20,66.20
4,MSP,2015-06-18 02:53,66.92,44.8854,-93.2313,66.92,66.92


In [10]:
missing_temps = raw_df[pd.isna(raw_df['temp'])]
missing_temps.shape

(10880084, 7)

In [16]:
raw_df['temp_backfill'] = raw_df['temp'].fillna(method='backfill', limit=1)

In [18]:
missing_temps = raw_df[pd.isna(raw_df['temp_backfill'])]
missing_temps.shape


(9627182, 8)

In [93]:
s = pd.Series(range(24*4+1), index=pd.date_range('2020-01-01', '2020-01-02', freq='15min'))
s.rolling(window='1H').mean()

2020-01-01 00:00:00     0.0
2020-01-01 00:15:00     0.5
2020-01-01 00:30:00     1.0
2020-01-01 00:45:00     1.5
2020-01-01 01:00:00     2.5
                       ... 
2020-01-01 23:00:00    90.5
2020-01-01 23:15:00    91.5
2020-01-01 23:30:00    92.5
2020-01-01 23:45:00    93.5
2020-01-02 00:00:00    94.5
Freq: 15T, Length: 97, dtype: float64

In [143]:
from weather_data import ASOS
asos = ASOS()
dlh_df = asos.get_station_df('DLH', est(2016, 6, 16, 0), est(2016, 6, 18, 23))
dlh_df[12:30]

,station,time,tmpf
valid,,,
2016-06-18 23:55:00+00:00,DLH,2016-06-18 23:55:00+00:00,78.08
2016-06-18 23:55:00+00:00,DLH,2016-06-18 23:55:00+00:00,78.08
2016-06-18 23:55:00+00:00,DLH,2016-06-18 23:55:00+00:00,78.08
2016-06-18 23:55:00+00:00,DLH,2016-06-18 23:55:00+00:00,78.08
2016-06-18 23:55:00+00:00,DLH,2016-06-18 23:55:00+00:00,78.08
2016-06-18 23:55:00+00:00,DLH,2016-06-18 23:55:00+00:00,78.08
2016-06-18 23:55:00+00:00,DLH,2016-06-18 23:55:00+00:00,78.08
2016-06-18 23:55:00+00:00,DLH,2016-06-18 23:55:00+00:00,78.08
2016-06-18 23:55:00+00:00,DLH,2016-06-18 23:55:00+00:00,78.08


In [194]:
from weather_data import ASOS
asos = ASOS()
dlh_df = asos.get_station_df('DLH', est(2016, 6, 16, 0), est(2016, 6, 18, 23))
dlh_df[12:30]

,station,tmpf,observing_station,observation_time
idx,,,,
2016-06-17 01:00:00+00:00,DLH,69.08,DLH,2016-06-17 00:55:00+00:00
2016-06-17 03:00:00+00:00,DLH,59.00,DLH,2016-06-17 02:55:00+00:00
2016-06-17 06:00:00+00:00,DLH,55.94,DLH,2016-06-17 05:55:00+00:00
2016-06-17 07:00:00+00:00,DLH,55.04,DLH,2016-06-17 06:55:00+00:00
2016-06-17 08:00:00+00:00,DLH,55.04,DLH,2016-06-17 07:55:00+00:00
2016-06-17 09:00:00+00:00,DLH,55.04,DLH,2016-06-17 08:55:00+00:00
2016-06-17 12:00:00+00:00,DLH,57.92,DLH,2016-06-17 11:55:00+00:00
2016-06-17 14:00:00+00:00,DLH,66.02,DLH,2016-06-17 13:55:00+00:00
2016-06-17 15:00:00+00:00,DLH,71.06,DLH,2016-06-17 14:55:00+00:00


In [202]:
from weather_data import ASOS
asos = ASOS()
dlh_df = asos.get_station_df('DLH', est(2016, 6, 16, 0), est(2016, 6, 18, 23))
dlh_df[12:30]

,station,tmpf,observing_station,observation_time
2016-06-16 17:00:00+00:00,DLH,65.445070,DLH,2016-06-16 16:55:00+00:00
2016-06-16 18:00:00+00:00,DLH,67.296671,DLH,2016-06-16 18:00:00+00:00
2016-06-16 19:00:00+00:00,DLH,69.237073,DLH,2016-06-16 19:00:00+00:00
2016-06-16 20:00:00+00:00,DLH,70.668092,DLH,2016-06-16 20:00:00+00:00
2016-06-16 21:00:00+00:00,DLH,71.960000,DLH,2016-06-16 20:55:00+00:00
2016-06-16 22:00:00+00:00,DLH,71.960282,DLH,2016-06-16 22:05:00+00:00
2016-06-16 23:00:00+00:00,DLH,71.965029,DLH,2016-06-16 22:55:00+00:00
2016-06-17 00:00:00+00:00,DLH,71.485115,DLH,2016-06-17 00:05:00+00:00
2016-06-17 01:00:00+00:00,DLH,66.639983,DLH,2016-06-17 01:05:00+00:00
2016-06-17 02:00:00+00:00,DLH,62.992230,DLH,2016-06-17 01:55:00+00:00


In [187]:
dlh_df.groupby('observing_station').count()

,station,tmpf,observation_time
observing_station,,,
DLH,49,49,49
DYT,17,17,17
SUW,1,1,1


In [186]:
dlh_df[pd.isna(dlh_df['tmpf'])]

,station,tmpf,observing_station,observation_time
2016-06-18 00:00:00+00:00,DLH,NaN,NaN,NaT
2016-06-19 01:00:00+00:00,DLH,NaN,NaN,NaT
2016-06-19 02:00:00+00:00,DLH,NaN,NaN,NaT
2016-06-19 03:00:00+00:00,DLH,NaN,NaN,NaT
2016-06-19 04:00:00+00:00,DLH,NaN,NaN,NaT


In [159]:
est(2020, 1, 1, 0)

datetime.datetime(2020, 1, 1, 0, 0, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=68400)))

In [103]:
dlh_df.groupby([dlh_df.index.date, dlh_df.index.hour]).tmpf.mean()

            valid
2016-06-16  4        49.640
            5        50.000
            6        50.504
            7        50.000
            8        50.000
            9        51.080
            10       51.620
            11       51.980
            12       50.660
            13       51.530
            14       55.904
            15       55.400
            16       56.480
            17       55.400
            18       57.200
            19       61.400
            20       71.960
            21       64.400
            22       67.415
            23       71.060
2016-06-17  0        63.050
            1        59.450
            2        59.000
            3        55.400
            4        55.400
            5        55.280
            6        55.040
            7        55.040
            8        55.040
            9        53.600
            10       53.600
            11       57.920
            12       53.600
            13       58.055
            14       71.060
  

In [91]:
from weather_data import ASOS
asos = ASOS()
dlh_df = asos.get_station_df('DLH', datetime(2016, 6, 16), datetime(2016, 6, 18))
dlh_df.head()

using DYT 2016-06-16 04:14:00+00:00 48.2 for DLH 2016-06-16 04:05:00+00:00
using DYT 2016-06-16 05:54:00+00:00 48.2 for DLH 2016-06-16 06:00:00+00:00
using DYT 2016-06-16 07:54:00+00:00 50.0 for DLH 2016-06-16 08:00:00+00:00
using DYT 2016-06-16 12:34:00+00:00 50.0 for DLH 2016-06-16 12:30:00+00:00
using DYT 2016-06-16 12:53:00+00:00 50.0 for DLH 2016-06-16 12:50:00+00:00
using DYT 2016-06-16 12:53:00+00:00 50.0 for DLH 2016-06-16 13:00:00+00:00
using DYT 2016-06-16 13:14:00+00:00 50.0 for DLH 2016-06-16 13:20:00+00:00
using DYT 2016-06-16 15:33:00+00:00 55.4 for DLH 2016-06-16 15:30:00+00:00
using DYT 2016-06-16 15:54:00+00:00 55.4 for DLH 2016-06-16 15:50:00+00:00
using DYT 2016-06-16 15:54:00+00:00 55.4 for DLH 2016-06-16 16:00:00+00:00
using DYT 2016-06-16 16:14:00+00:00 55.4 for DLH 2016-06-16 16:15:00+00:00
using DYT 2016-06-16 16:33:00+00:00 57.2 for DLH 2016-06-16 16:30:00+00:00
using DYT 2016-06-16 16:54:00+00:00 57.2 for DLH 2016-06-16 16:45:00+00:00
using DYT 2016-06-16 16:5

,station,tmpf,time
valid,,,
2016-06-16 04:05:00+00:00,DLH,48.2,2016-06-16 04:05:00+00:00
2016-06-16 04:15:00+00:00,DLH,50.0,2016-06-16 04:15:00+00:00
2016-06-16 04:35:00+00:00,DLH,50.0,2016-06-16 04:35:00+00:00
2016-06-16 04:45:00+00:00,DLH,50.0,2016-06-16 04:45:00+00:00
2016-06-16 04:55:00+00:00,DLH,50.0,2016-06-16 04:55:00+00:00


In [92]:
missing = dlh_df[pd.isna(dlh_df['tmpf'])]
missing.head(20)

,station,tmpf,time
valid,,,


We need a temperature observation for each hour, since that is how the MISO MTLF is reported. Our initial approach will be to simply drop all missing observations, then choose the observation that is closest in time to the top of the hour.

In [35]:
observation_dates = pd.date_range(start = first_hour, end = last_hour)
observation_hours = [d.replace(hour = h) for d in observation_dates for h in range(0, 24)]

def build_hourly_df(zone, state, station):
    w = pd.read_parquet(download_file_path(zone, state, station))
    df = w[w['tmpf'] != 'M'].copy()
    df['valid'] = pd.to_datetime(df['valid'], utc=True)
    numeric_cols = ['tmpf', 'lat', 'lon']
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, axis=1)
    df = df.drop(columns=['feel'])
    idx = df.drop_duplicates('valid').set_index('valid').index.get_indexer(observation_hours, method='nearest')
    df = df.iloc[idx]
    df.loc[: , 'valid'] = df['valid'].dt.round(freq='H')
    return df.drop_duplicates('valid')


In [36]:
dfs = do_parallel(build_hourly_df)
zonal_weather_data = {}
for zone in zones:
    zonal_weather_data[zone] = pd.concat(dfs[zone], ignore_index=True)

In [37]:
df = zonal_weather_data[1]
for station in pd.unique(zonal_weather_data[1]['station']):
    count = df[df['station'] == station].shape[0]
    print(f'{station} observation count {count}')
print(f'Required observations {len(observation_hours)}')

MSP observation count 59898
RST observation count 59820
DLH observation count 49334
FAR observation count 59806
BIS observation count 59792
GFK observation count 59780
ABR observation count 59714
LSE observation count 59456
SFY observation count 57519
Required observations 60024


In [44]:
merged = pd.merge(df[df['station'] == 'DLH'], pd.DataFrame(observation_hours, columns=['Hours']), how='right', left_on='valid', right_on='Hours')
merged[merged['station'].isna()]['Hours'].apply(lambda h: h.date).value_counts()[0:50]

2021-08-22    24
2020-10-30    23
2020-09-23    22
2018-09-12    21
2021-05-02    21
2018-08-22    20
2020-10-25    19
2021-05-03    18
2018-04-25    17
2019-07-07    17
2018-06-20    17
2018-05-14    17
2018-05-07    17
2018-07-30    17
2018-05-10    17
2018-07-14    17
2020-07-20    16
2018-07-06    16
2019-03-25    16
2018-03-22    16
2018-07-11    16
2018-03-21    16
2018-07-13    16
2020-02-06    16
2019-04-27    16
2018-07-17    16
2019-06-13    16
2018-03-17    16
2018-03-16    16
2018-07-05    16
2018-03-13    16
2018-07-23    16
2018-07-24    16
2018-07-27    16
2018-03-09    16
2018-03-14    16
2018-06-19    16
2019-06-20    16
2019-07-16    16
2018-05-11    16
2018-05-12    16
2018-05-13    16
2019-07-31    16
2018-05-15    16
2018-05-06    16
2018-05-18    16
2018-05-05    16
2018-05-20    16
2018-05-22    16
2019-07-13    16
Name: Hours, dtype: int64

There are entire days missing in the observations. We need to try to obtain observations from nearby stations to fill in the lacunae.

In [39]:
hour = merged[merged['station'].isna()].iloc[0]['Hours']
from weather_data import get_nearest_observation, get_session
tmpf = get_nearest_observation('DLH', 'MN', hour, get_session())

Found 105 valid sites for MN
Pandas(Index=0, elevation=367.0, sname='AITKIN NDB', time_domain='(1991-Now)', state='MN', country='US', climate_site='MN0059', wfo='DLH', tzname='America/Chicago', ncdc81='USC00210059', ncei91='USC00210059', ugc_county='MNC001', ugc_zone='MNZ036', county='Aitkin', sid='AIT', latlon=(46.5484, -93.6768), distance=73.35199477929798)


In [1]:
df.head()

NameError: name 'df' is not defined

In [ ]:
import pandas as pd
from weather_data import get_stations, get_session
duluth = pd.read_parquet(download_file_path(1, 'MN', 'DLH'))
stations_raw = get_stations('MN', 2015, get_session('https://'))

In [ ]:
# sort stations MN by closest to another station
from geopy import distance
dlh = stations_MN_df[stations_MN_df['sid'] == 'DLH'].iloc[0]['latlon']
stations_MN_df['distance_from_DLH'] = stations_MN_df['latlon'].apply(lambda c: distance.distance(dlh, c).mi)

In [ ]:
stations_MN_df.sort_values(['distance_from_DLH'])

In [ ]:
distance.distance(dlh_latlon, dlh_latlon)

### Obtain the Regional MTLF and Actual Load for each Observation Hour

In [ ]:
from rf_al_data import 

Path("./data/mtlf").mkdir(exist_ok=True)
forecast_output_dir = './data/mtlf'
# the actuals aren't available until the next day
actuals = get_daily_rf_al_df(first_hour, last_hour + timedelta(days=2.0), forecast_output_dir)
actuals

### Harmonize Features with Actuals

There are a number of lacunae in the weather observations.

In [ ]:
def mktime_idx(row): 
    return datetime.combine(row['Market Day'].date(), time(row['HourEnding'] - 1), timezone(timedelta(hours = -5)))

actuals['time_idx'] = actuals.apply(mktime_idx, axis = 1)

In [ ]:
actuals.to_parquet('./data/actuals_mtlf.parquet')
actuals

In [ ]:
p = df.pivot(index='valid', columns='station', values='tmpf').dropna()
data = p.join(actuals.set_index('time_idx'), how='inner')

(n, _) = data.shape
(num_weather_observations, _) = df.groupby('valid').count().shape
(num_load_observations, _) = actuals.shape
(num_weather_observations, num_load_observations, len(observation_hours), n)

## Feature Engineering: Business Hours

Can we improve the performance of the model by introducing business hours into the feature set?

In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay, BusinessHour

federal_business_days = CustomBusinessDay(calendar=USFederalHolidayCalendar())
bh = BusinessHour()
def is_biz_hour(d):
    return federal_business_days.is_on_offset(d) and bh.is_on_offset(d)
data['IsBusinessHour'] = data.index.to_series().apply(lambda d: 1 if is_biz_hour(d) else 0)